# League of Legends Project

## Part 1: Data Collection
    Ranked match data will be collected from each Summoner Name provided. 

    Part 1A: Data Collection from Data Dragon
        - create mapping dictionaries from the json files provided

    Part 1B: Data Collection from Riot API
        - extract data from RIOT API
    
## Part 2: Data Analysis & Machine Learning
    Exploratory Data Analysis will be done on the match data to discover basic statistics and trends.
    
    Part 2A: Data Wrangling and Cleaning
        - wrangle and manipulate data table for consistency
        - prepare data for visualization and machine learning

    Part 2B: Data Visualization
        - visualize trends and correlations show in the data
        - create dashboard to visualize game statistics by player
        
    Part 2C: Machine Learning
        - using data extracted from top 100 players, predict the outcome of a match based off the different variables provided
        - using data extracted from top 100 players, identify and group champions based off classification algorithm

# Part 1A

In [1]:
import pandas as pd
import plotly.express as px
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import regex as re
import json
import pprint
import time
from bs4 import BeautifulSoup

## Champion JSON

In [35]:
def create_champion_mapping_dict(file_name):
    champion_key_dict = {}
    with open(f'json_files/{file_name}', encoding = 'utf-8') as f:
        champion_json = json.load(f)
        for champion, info in champion_json['data'].items():
            champion_name = champion
            for c_key, c_value in info.items():
                if c_key == 'key':
                    champion_key_dict[champion_name] = int(c_value)
    return champion_key_dict

champion_key_dict = create_champion_mapping_dict('champion.json')

## Item JSON

In [36]:
def create_item_mapping_dict(file_name):
    item_key_dict = {}
    with open(f'json_files/{file_name}', encoding = 'utf-8') as f:
        item_data = json.load(f)
        for item, info in item_data['data'].items():
            item_id = item
            for i_key, i_value in info.items():
                if i_key == 'name':
                    item_key_dict[int(item_id)] = i_value
    return item_key_dict

item_key_dict = create_item_mapping_dict('item.json')

## Queue JSON

In [37]:
def create_queue_mapping_dict(file_name):
    queue_info_dict = {}
    with open(f'json_files/{file_name}', encoding = 'utf-8') as f:
        queue_data = json.load(f)
        for queue in queue_data:
            queue_key = queue['queueId']
            queue_name = queue['description']
            queue_notes = str(queue['notes'])
            if 'deprecated' not in queue_notes.lower():
                queue_info_dict[queue_key] = str(queue_name).upper()

    queue_info_dict_reverse = {str(v).upper(): k for (k,v) in queue_info_dict.items()}
    return queue_info_dict, queue_info_dict_reverse

queue_key_dict, queue_key_dict_reverse = create_queue_mapping_dict('queue.json')

## Summoner Spell JSON

In [38]:
def create_summoner_spell_mapping_dict(file_name):
    ss_info_dict = {}
    with open(f'json_files/{file_name}', encoding = 'utf-8') as f:
        ss_data = json.load(f)
        for s_spell, info in ss_data['data'].items():
            for key in info:
                ss_key = info['key']
                ss_name = info['name']
                ss_description = info['description']
                ss_info_dict[ss_name] = (ss_key, ss_description)
    return ss_info_dict

ss_key_dict = create_summoner_spell_mapping_dict('summoner.json')

## Season JSON

In [39]:
def create_season_mapping_dict(file_name):
    season_info_dict = {}
    with open(f"json_files/{file_name}", encoding = "utf-8") as f:
        season_data = json.load(f)
        for season in season_data:
            season_info_dict[season['id']] = season['season']
    return season_info_dict
    
season_key_dict = create_season_mapping_dict('season.json')

# Part 1B

In [40]:
REQUEST_HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.66 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Charset': 'application/x-www-form-urlencoded; charset=UTF-8',
    'Origin': 'https://developer.riotgames.com',
    'X-Riot-Token': ''
    }
RIOT_API = ''

## Getting Account Information Using Summoner Name

In [41]:
def get_puu_id(summoner_name, riot_api_key = RIOT_API, request_headers = REQUEST_HEADERS, access = True):
    summoner_name_updated = summoner_name.replace(' ', '%20')
    url = 'https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/'
    account_url = url + summoner_name_updated
    riot_headers = request_headers
    riot_headers["X-Riot-Token"] = riot_api_key
    while access:
        req = requests.get(account_url, headers = riot_headers)
        if req.status_code == 200:
            account_info = json.loads(req.content.decode("utf-8"))
            puu_id = account_info['puuid']
            access = False

        elif req.status_code == 404:
            puu_id = None
            access = False

        else:
            time.sleep(10)
    return  puu_id

## Getting A List of Match IDs Using PUU ID

In [42]:
def get_match_ids(puu_id, start, end, riot_api_key = RIOT_API, request_headers = REQUEST_HEADERS, access = True):
    match_ids = []
    match_history_url = f'https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puu_id}/ids?start={start}&count={end}'
    riot_headers = request_headers
    riot_headers["X-Riot-Token"] = riot_api_key

    while access:
        req = requests.get(match_history_url, headers = riot_headers)
        if req.status_code == 200:
            match_id_info = json.loads(req.content.decode("utf-8"))
            for match_id in match_id_info:
                match_ids.append(match_id)
            access = False
        else:
            print('Waiting for 10 seconds to refresh restriction request...')
            time.sleep(10)

    return match_ids

## Get Match Information Using Match ID

In [43]:
def get_match_details(match_id, riot_api_key = RIOT_API, request_headers = REQUEST_HEADERS, access = True):
    match_url = 'https://americas.api.riotgames.com/lol/match/v5/matches/' + str(match_id)
    riot_headers = request_headers
    riot_headers["X-Riot-Token"] = riot_api_key

    while access:
        req = requests.get(match_url, headers = riot_headers)
        if req.status_code == 200:
            match_details = json.loads(req.content.decode("utf-8"))
            access = False

        elif req.status_code == 404:
            print(f'{match_id} - DATA NOT FOUND - MATCH FILE NOT FOUND')
            match_details = None
            access = False
            
        else:
            print(req.status_code)
            print('Waiting for 10 seconds to refresh restriction request...')
            time.sleep(10)

    return match_details

## Cleaning and Reducing Match Data

In [44]:
def clean_and_reduce_match_data(match_details, queue_type, queue_mapping_info = queue_key_dict_reverse):
    queue_filter = queue_mapping_info[queue_type.upper()]
    match_cleaned_dict = {}
    for key, value in match_details['info'].items():

        if key == 'gameId':
            match_id = value

        if key == 'gameMode':
            match_type = value
            
        if key == 'participants':
            participant_details = value
        
        if key == 'queueId':
            if value == queue_filter:
                queue_id = value
                match_cleaned_dict[(match_type, queue_id, match_id)] = participant_details
            else:
                return None
    return match_cleaned_dict

In [45]:
def filter_match_data_by_puuid(match_cleaned_dict, puu_id):
    for match_id, match_details in match_cleaned_dict.items():
        for participant_dict in match_details:
            if participant_dict['puuid'] == puu_id:
                return {match_id[0] + '_' + str(match_id[1]) + '_' + str(match_id[2]): participant_dict}

In [46]:
def remove_perks_attribute(match):
    for key, info in match.items():
        for key in list(info.keys()):
            if key == 'perks':
                del info[key]

    return match

## Turn Match Data into DataFrame

In [47]:
def make_match_df(match):
    match_df = pd.DataFrame.from_dict(match, orient = 'index')
    return match_df

## Create Super Function for Data Collection

In [48]:
def get_match_history_by_summoner_name(summoner_name, start, end, queue_type, riot_api = RIOT_API, request_headers = REQUEST_HEADERS):
    puu_id = get_puu_id(summoner_name, riot_api, request_headers)

    if puu_id != None:
        match_ids = get_match_ids(puu_id, start, end, riot_api, request_headers)

        matches_df_listed = []

        for match_id in match_ids:
            match_detail = get_match_details(match_id, riot_api, request_headers)
            if match_detail != None:
                match_cleaned = clean_and_reduce_match_data(match_detail, queue_type)
                if match_cleaned != None:
                    match_cleaned_filtered = filter_match_data_by_puuid(match_cleaned, puu_id)

                    match_final = remove_perks_attribute(match_cleaned_filtered)

                    match_df = make_match_df(match_final)
    
                    matches_df_listed.append(match_df)
                    
                else:
                    print(f'Match ID: {match_id} IS NOT OF THE QUEUE TYPE: {queue_type.upper()}')
                    continue

        if matches_df_listed != []:
            return pd.concat(matches_df_listed)
                
        
    else:
        return f'NO PUU_ID FOUND FOR SUMMONER: {summoner_name}'

# Part 2A

## Create Super Function for Initial Data Cleanup


In [49]:
def clean_columns(data):
    data = data.reset_index()

    cleaned_cols = list(data.columns.values)[1:]

    data[['matchType', 'queueId', 'matchId']] = data['index'].str.split('_', expand = True)

    cleaned_cols = ['queueId', 'matchId'] + cleaned_cols
    data = data[cleaned_cols]
    
    data['queueId'] = data['queueId'].astype(int)

    return data

def mapping_data_values(data, mapping_key):
    if mapping_key == 'Q':
        data['queueId'] = data['queueId'].map(queue_key_dict)

    if mapping_key == 'I':
        item_columns = ["item0", "item1", "item2", "item3", "item4", "item5", "item6"]
        for col in item_columns:
            data[col] = data[col].map(item_key_dict)

    if mapping_key == 'S':
        summoner_columns = ['summoner1Id', 'summoner2Id']
        for col in summoner_columns:
            data[col] = data[col].map(ss_key_dict)
    
    if mapping_key == 'T':
        team_color = data['teamId'].map({100: 'Blue', 200: 'Red'})
        data.insert(1, 'teamColor', team_color)
    
    return data

In [50]:
def initial_cleanup(data):
    data = clean_columns(data)

    data = mapping_data_values(data, 'Q')

    data = mapping_data_values(data, 'I')

    data = mapping_data_values(data, 'S')

    data = mapping_data_values(data, 'T')

    return data

## Combine All Super Functions

In [51]:
def get_clean_match_data_by_summoner(summoner_name, start, end, queue_type):

    df = get_match_history_by_summoner_name(summoner_name, start, end, queue_type)
    if isinstance(df, pd.DataFrame):
        df = initial_cleanup(df)
    else:
        df = f'NO MATCHES OF THE TYPE {queue_type.upper()} IN THE LAST {end} GAMES'   

    return df

In [52]:
def get_summoner_match_clean_df(summoner_name, num_games, q_type):

    solo_game_count = 0
    starting = 101
    df_listed = []
    ending = 1

    print(f'STARTED PROCESSING FOR SUMMONER: {summoner_name}')
    print(f'ROW {starting} HAS BEEN PROCESSED')

    while solo_game_count < num_games:
        df = get_clean_match_data_by_summoner(summoner_name = summoner_name, start = starting, end = ending, queue_type = q_type)
        starting += ending
        print(f'ROW {starting} HAS BEEN PROCESSED')

        if isinstance(df, pd.DataFrame):
            solo_game_count += df.shape[0]
            print(f'SOLO GAME ROW COUNT: {solo_game_count}')
            df_listed.append(df)

    final = pd.concat(df_listed)
    return final

## Getting Most Recent 50 Solo Ranked Matches From Top 100 Challenger Level Players

In [53]:
def get_top_100_challenger_players(chall_url):
    response = requests.get(chall_url)
    html_soup  = BeautifulSoup(response.text, 'html.parser')
    challlenger_top_100_snames = []

    chall_top_4 = html_soup.find_all('li', class_ = 'ranking-highest__item')
    for challenger in chall_top_4:
        chall_sname = challenger.find('a', class_ = 'ranking-highest__name').text
        challlenger_top_100_snames.append(chall_sname)
    
    chall_rest = html_soup.find_all('tr', class_ = 'ranking-table__row')
    for challenger in chall_rest:
        challlenger_top_100_snames.append(challenger.span.text)

    return challlenger_top_100_snames

In [54]:
challenger_top100_snames = get_top_100_challenger_players('https://na.op.gg/ranking/ladder/page=1')

In [55]:
match_df_listed = []
for challenger in challenger_top100_snames:
    match_df = get_summoner_match_clean_df(summoner_name = challenger, num_games = 50, q_type = '5v5 Ranked Solo Games')
    match_df_listed.append(match_df)

STARTED PROCESSING FOR SUMMONER: From Iron
ROW 101 HAS BEEN PROCESSED
ROW 102 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 1
ROW 103 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 2
ROW 104 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 3
ROW 105 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 4
ROW 106 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 5
ROW 107 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 6
ROW 108 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 7
ROW 109 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 8
ROW 110 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 9
ROW 111 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 10
ROW 112 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 11
ROW 113 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 12
ROW 114 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 13
ROW 115 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 14
ROW 116 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 15
ROW 117 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 16
ROW 118 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 17
ROW 119 HAS BEEN PROCESSED
SOLO GAME ROW COUNT: 18
ROW 120 HAS BEEN PROC

In [56]:
final_df = pd.concat(match_df_listed)
final_df.head()

,queueId,teamColor,matchId,assists,baronKills,bountyLevel,champExperience,champLevel,championId,championName,championTransform,consumablesPurchased,damageDealtToBuildings,damageDealtToObjectives,damageDealtToTurrets,damageSelfMitigated,deaths,detectorWardsPlaced,doubleKills,dragonKills,firstBloodAssist,firstBloodKill,firstTowerAssist,firstTowerKill,gameEndedInEarlySurrender,gameEndedInSurrender,goldEarned,goldSpent,individualPosition,inhibitorKills,inhibitorsLost,item0,item1,item2,item3,item4,item5,item6,itemsPurchased,killingSprees,kills,lane,largestCriticalStrike,largestKillingSpree,largestMultiKill,longestTimeSpentLiving,magicDamageDealt,magicDamageDealtToChampions,magicDamageTaken,neutralMinionsKilled,nexusKills,nexusLost,objectivesStolen,objectivesStolenAssists,participantId,pentaKills,physicalDamageDealt,physicalDamageDealtToChampions,physicalDamageTaken,profileIcon,puuid,quadraKills,riotIdName,riotIdTagline,role,sightWardsBoughtInGame,spell1Casts,spell2Casts,spell3Casts,spell4Casts,summoner1Casts,summoner1Id,summoner2Casts,summoner2Id,summonerId,summonerLevel,summonerName,teamEarlySurrendered,teamId,teamPosition,timeCCingOthers,timePlayed,totalDamageDealt,totalDamageDealtToChampions,totalDamageShieldedOnTeammates,totalDamageTaken,totalHeal,totalHealsOnTeammates,totalMinionsKilled,totalTimeCCDealt,totalTimeSpentDead,totalUnitsHealed,tripleKills,trueDamageDealt,trueDamageDealtToChampions,trueDamageTaken,turretKills,turretsLost,unrealKills,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win
0,5V5 RANKED SOLO GAMES,Red,3927462189,5,0,0,5969,9,235,Senna,0,5,1116.0,1319,1116,5158,5,1,0,0,False,False,False,False,False,True,6150,5875,UTILITY,0,1.0,Kraken Slayer,Refillable Potion,Black Mist Scythe,Control Ward,Berserker's Greaves,Rageknife,Oracle Lens,17,0,2,BOTTOM,293,0,1,574,0,0,2927,0,0,0.0,0,0,10,0,32646,10180,7247,4568,6zpHc103eNNhAt8xC6Ujhak476wF7TuTm4K4vrCU2Zuu9Y...,0,,,SUPPORT,0,44,25,15,4,3,NaN,4,NaN,-wR2sAu8IN0jiLgjgGyDQsjpnyX5tSwTwZHcIE7L9aS1TbyQ,268,From Iron,False,200,UTILITY,26,1212,34587,11018,607,10646,6417,3029,24,113,116,6,0,1941,838,472,0,6.0,0,28,3,5,15,False
0,5V5 RANKED SOLO GAMES,Red,3927346919,10,0,0,10327,13,111,Nautilus,0,12,487.0,738,487,14240,4,9,0,0,True,False,False,False,False,False,7115,6475,UTILITY,0,2.0,Plated Steelcaps,Locket of the Iron Solari,Bulwark of the Mountain,NaN,Kindlegem,Glacial Buckler,Oracle Lens,26,0,1,BOTTOM,0,0,1,1096,9882,3923,3068,2,0,1.0,0,0,10,0,5165,681,6704,4568,6zpHc103eNNhAt8xC6Ujhak476wF7TuTm4K4vrCU2Zuu9Y...,0,,,SUPPORT,0,54,30,27,11,9,NaN,5,NaN,-wR2sAu8IN0jiLgjgGyDQsjpnyX5tSwTwZHcIE7L9aS1TbyQ,268,From Iron,False,200,UTILITY,35,1940,20605,5124,543,11593,879,396,25,235,97,5,0,5557,520,1820,0,10.0,0,76,10,5,40,False
0,5V5 RANKED SOLO GAMES,Red,3927028042,23,0,0,12463,14,89,Leona,0,13,438.0,1387,438,25987,2,7,0,0,True,False,False,False,False,False,9501,8775,UTILITY,0,1.0,Mercury's Treads,Locket of the Iron Solari,Pauldrons of Whiterock,Control Ward,Abyssal Mask,Bramble Vest,Oracle Lens,33,1,3,BOTTOM,0,3,1,869,8075,3186,6097,0,0,0.0,0,0,10,0,6593,1847,10116,4568,6zpHc103eNNhAt8xC6Ujhak476wF7TuTm4K4vrCU2Zuu9Y...,0,,,SUPPORT,0,48,35,25,10,11,NaN,4,NaN,-wR2sAu8IN0jiLgjgGyDQsjpnyX5tSwTwZHcIE7L9aS1TbyQ,268,From Iron,False,200,UTILITY,35,1966,20458,5431,888,17543,3966,406,42,56,72,5,0,5789,396,1329,0,7.0,0,100,10,16,38,True
0,5V5 RANKED SOLO GAMES,Blue,3927064248,18,0,0,7212,10,111,Nautilus,0,9,138.0,138,138,12752,2,5,0,0,True,False,False,False,False,True,6641,6425,UTILITY,0,0.0,Plated Steelcaps,Locket of the Iron Solari,Bulwark of the Mountain,Control Ward,Giant's Belt,Negatron Cloak,Oracle Lens,21,0,2,NONE,0,0,1,398,8736,3595,2446,0,0,0.0,0,0,5,0,7307,2145,6909,4568,6zpHc103eNNhAt8xC6Ujhak476wF7TuTm4K4vrCU2Zuu9Y...,0,,,SUPPORT,0,45,23,18,4,10,NaN,5,NaN,-wR2sAu8IN0jiLgjgGyDQsjpnyX5tSwTwZHcIE7L9aS1TbyQ,268,From Iron,False,100,UTILITY,46,1159,22190,6265,126,9774,1332,677,29,195,41,5,0,6146,524,418,0,1.0,0,45,7,4,21,True
0,5V5 RANKED SOLO GAMES,Red,3927019948,7,0,0,6413,

In [57]:
final_df.shape

(5000, 104)

In [67]:
final_df.to_csv('export_data/data_6_18.csv', index = False)

# Part 2B

In [2]:
match_data_18 = pd.read_csv('export_data/data_6_18.csv')
match_data_17 = pd.read_csv('export_data/data_6_17.csv')
match_data_15 = pd.read_csv('export_data/data_6_15.csv')

In [3]:
match_data_total = pd.concat([match_data_15, match_data_17, match_data_18])

In [4]:
match_data_total.head()

,queueId,teamColor,matchId,assists,baronKills,bountyLevel,champExperience,champLevel,championId,championName,championTransform,consumablesPurchased,damageDealtToBuildings,damageDealtToObjectives,damageDealtToTurrets,damageSelfMitigated,deaths,detectorWardsPlaced,doubleKills,dragonKills,firstBloodAssist,firstBloodKill,firstTowerAssist,firstTowerKill,gameEndedInEarlySurrender,gameEndedInSurrender,goldEarned,goldSpent,individualPosition,inhibitorKills,inhibitorsLost,item0,item1,item2,item3,item4,item5,item6,itemsPurchased,killingSprees,kills,lane,largestCriticalStrike,largestKillingSpree,largestMultiKill,longestTimeSpentLiving,magicDamageDealt,magicDamageDealtToChampions,magicDamageTaken,neutralMinionsKilled,nexusKills,nexusLost,objectivesStolen,objectivesStolenAssists,participantId,pentaKills,physicalDamageDealt,physicalDamageDealtToChampions,physicalDamageTaken,profileIcon,puuid,quadraKills,riotIdName,riotIdTagline,role,sightWardsBoughtInGame,spell1Casts,spell2Casts,spell3Casts,spell4Casts,summoner1Casts,summoner1Id,summoner2Casts,summoner2Id,summonerId,summonerLevel,summonerName,teamEarlySurrendered,teamId,teamPosition,timeCCingOthers,timePlayed,totalDamageDealt,totalDamageDealtToChampions,totalDamageShieldedOnTeammates,totalDamageTaken,totalHeal,totalHealsOnTeammates,totalMinionsKilled,totalTimeCCDealt,totalTimeSpentDead,totalUnitsHealed,tripleKills,trueDamageDealt,trueDamageDealtToChampions,trueDamageTaken,turretKills,turretsLost,unrealKills,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win
0,5V5 RANKED SOLO GAMES,Blue,3944288481,13,0,0,8243,11,223,TahmKench,0,7,488.0,1419,488,17878,6,3,0,0,False,False,False,False,False,True,7039,6425,BOTTOM,0,0.0,Doran's Shield,Sunfire Aegis,Plated Steelcaps,Kindlegem,Ruby Crystal,NaN,Stealth Ward,18,0,1,BOTTOM,0,0,1,312,20774,4743,6487,0,0,0.0,0,0,5,0,10610,731,9696,4568,6zpHc103eNNhAt8xC6Ujhak476wF7TuTm4K4vrCU2Zuu9Y...,0,NaN,NaN,CARRY,0,71,4,11,2,3,NaN,4,NaN,-wR2sAu8IN0jiLgjgGyDQsjpnyX5tSwTwZHcIE7L9aS1TbyQ,274,From Iron,False,100,BOTTOM,14,1284,31384,5475,1105,16532,2087,0,98,105,140,1,0,0,0,349,0,0.0,0,16,3,4,9,True
1,5V5 RANKED SOLO GAMES,Blue,3944251280,11,0,0,6578,10,111,Nautilus,0,10,0.0,174,0,16060,5,7,0,0,False,False,False,False,False,False,6099,6000,UTILITY,0,1.0,Plated Steelcaps,Locket of the Iron Solari,Bulwark of the Mountain,Cloth Armor,Warden's Mail,Control Ward,Oracle Lens,28,1,2,BOTTOM,0,2,1,407,10589,4457,4313,0,0,1.0,0,0,5,0,5748,1951,6472,4568,6zpHc103eNNhAt8xC6Ujhak476wF7TuTm4K4vrCU2Zuu9Y...,0,NaN,NaN,SUPPORT,0,50,17,21,9,13,NaN,4,NaN,-wR2sAu8IN0jiLgjgGyDQsjpnyX5tSwTwZHcIE7L9aS1TbyQ,274,From Iron,False,100,UTILITY,46,1347,21927,7026,671,11566,62,0,43,246,98,1,0,5589,616,779,0,9.0,0,53,8,6,31,False
2,5V5 RANKED SOLO GAMES,Red,3944226110,0,0,0,4583,8,202,Jhin,0,6,0.0,0,0,3309,5,2,0,0,False,False,False,False,False,True,4305,3100,BOTTOM,0,0.0,Serrated Dirk,Refillable Potion,Boots,Vampiric Scepter,NaN,Long Sword,Stealth Ward,16,0,1,NONE,358,0,1,281,148,0,1678,0,0,0.0,0,0,9,0,34032,3541,6081,4568,6zpHc103eNNhAt8xC6Ujhak476wF7TuTm4K4vrCU2Zuu9Y...,0,NaN,NaN,SUPPORT,0,36,17,3,10,2,NaN,3,NaN,-wR2sAu8IN0jiLgjgGyDQsjpnyX5tSwTwZHcIE7L9aS1TbyQ,274,From Iron,False,200,BOTTOM,3,912,34180,3541,0,8214,1718,216,99,7,75,3,0,0,0,454,0,3.0,0,18,2,3,10,False
3,5V5 RANKED SOLO GAMES,Red,3941904096,20,0,0,10417,13,111,Nautilus,0,11,320.0,999,320,30781,7,8,0,0,False,False,False,False,False,False,7821,6575,UTILITY,0,0.0,Plated Steelcaps,Locket of the Iron Solari,Bulwark of the Mountain,Control Ward,Kindlegem,Warden's Mail,Oracle Lens,27,0,0,BOTTOM,0,0,0,690,8449,5522,8327,0,0,0.0,0,0,10,0,7874,1919,10809,4568,6zpHc103eNNhAt8xC6Ujhak476wF7TuTm4K4vrCU2Zuu9Y...,0,NaN,NaN,SUPPORT,0,71,46,24,9,5,NaN,5,NaN,-wR2sAu8IN0jiLgjgGyDQsjpnyX5tSwTwZHcIE7L9aS1TbyQ,274,From Iron,False,200,UTILITY,60,1841,21192,8025,1644,19624,1203,0,40,190,212,1,0,4868,584,487,0,1.0,0,64,9,8,35,True
4,5V5 RANKED SOLO GAMES,Red,3941845033,18,0,1,8840,12,111,Nautilus,0,10,796.0,796,796,15092,4,6,0,0,False,False,False,

In [5]:
subset = ['puuid', 'kills', 'deaths','assists', 'totalDamageDealt', 'totalDamageTaken', 'totalHeal', 'item0', 'item1', 'item2', 'item3', 'item4', 'item5', 'item6']
match_data_total_deduped = match_data_total.drop_duplicates(subset, keep = 'first')
match_data_total_deduped.shape

(12282, 104)

In [6]:
duplicated_matches = match_data_total_deduped.groupby(['puuid', 'kills', 'deaths','assists', 'totalDamageDealt', 'totalDamageTaken', 'totalHeal', 'item0', 'item1', 'item2', 'item3', 'item4', 'item5', 'item6']).size().reset_index().rename(columns={0:'duplicateCount'})
duplicated_matches.loc[duplicated_matches['duplicateCount'] > 1, ].shape

(0, 15)

## Questions to Answer:

### Who is the most played champion by lane?
    - select the top 3 played champions by lane and display the # of games played
    

### Who are the top 3 champions that hold the highest win rate by lane?
    - visualize win rates by lane and visualize win rate by champions
    - to decrease bias from lack of games played, the champions that have at least average game count will only be used when looking at win rate

### Is there a correlation between damage dealt and winning?

In [7]:
test = match_data_total_deduped.groupby(['teamPosition', 'championName'])['matchId'].agg('count').reset_index()
test.rename(columns = {'matchId':'matchCount'}, inplace = True)
test_sorted = test.sort_values(['teamPosition', 'matchCount'], ascending = [True, False])
test_filtered = test_sorted.loc[test_sorted['teamPosition'] != 'AFK']
test_filtered
top3_bottom = ['Ezreal', 'Kaisa', 'Tristana']
top3_jungle = ['Kindred', 'Kayn', 'Rumble']
top3_mid = ['Sylas', 'LeeSin', 'Lucian']
top3_top = ['LeeSin', 'Jayce', 'Gwen']
top3_supp = ['Thresh', 'Nautilus', 'Karma']

bot_mask = test_sorted['teamPosition'] == 'BOTTOM'
jung_mask = test_sorted['teamPosition'] == 'JUNGLE'
mid_mask = test_sorted['teamPosition'] == 'MIDDLE'
top_mask = test_sorted['teamPosition'] == 'TOP'
supp_mask = test_sorted['teamPosition'] == 'UTILITY'

top3_all_lanes = test.loc[
    (bot_mask & test['championName'].isin(top3_bottom)) |
    (jung_mask & test['championName'].isin(top3_jungle)) |
    (mid_mask & test['championName'].isin(top3_mid)) |
    (top_mask & test['championName'].isin(top3_top)) |
    (supp_mask & test['championName'].isin(top3_supp))
]
top3_all_lanes_sorted = top3_all_lanes.sort_values(['teamPosition', 'matchCount'], ascending = [True, False])
top3_all_lanes_sorted.loc[top3_all_lanes_sorted['teamPosition'] == 'UTILITY', 'teamPosition'] = 'SUPPORT'
top3_all_lanes_sorted

,teamPosition,championName,matchCount
12,BOTTOM,Ezreal,344
19,BOTTOM,Kaisa,296
43,BOTTOM,Tristana,249
76,JUNGLE,Kindred,248
92,JUNGLE,Rumble,179
74,JUNGLE,Kayn,157
179,MIDDLE,Sylas,244
150,MIDDLE,LeeSin,208
152,MIDDLE,Lucian,177
231,TOP,LeeSin,237


In [8]:
fig = px.bar(top3_all_lanes_sorted, x = 'teamPosition', y = 'matchCount', color = 'championName', text = 'matchCount',
             color_discrete_map = {
                'Ezreal': '#5F9EA0',
                'Tristana': '#FFA07A',
                'Kaisa': '#4B0082',
                'Rumble': '#CD853F',
                'Kindred': '#708090',
                'Kayn': '#4682B4',
                'Lucian': '#DCDCDC',
                'LeeSin': '#A0522D',
                'Sylas': '#191970',
                'Gwen': '#40E0D0',
                'Jayce': '#D2B48C',
                'Nautilus': '#2E8B57',
                'Thresh': '#556B2F',
                'Karma': '#FFB6C1'
            },

             labels = {'teamPosition': 'Position', 'matchCount': '# of Games Played', 'championName': 'Champion'}, title = 'Top 3 Champions Played by Position')
fig.update_layout(
    plot_bgcolor = 'rgb(0,0,0,0)',
    paper_bgcolor = 'rgb(0,0,0,0)',
    width = 1200,
    height = 800,
    title_x = 0.5,
    legend = dict(bgcolor = 'white', yanchor = 'top', y = 0.88, xanchor = 'right', x = 1.10),
    legend_title = dict(font = dict(size = 14)),
)
fig.update_xaxes(categoryorder = 'total ascending')

fig.layout.legend.tracegroupgap = 14

fig.update_traces(texttemplate = '%{text} Games', textposition = 'inside', insidetextanchor = 'middle')

fig.show()

In [9]:
match_data_total_deduped.head()

,queueId,teamColor,matchId,assists,baronKills,bountyLevel,champExperience,champLevel,championId,championName,championTransform,consumablesPurchased,damageDealtToBuildings,damageDealtToObjectives,damageDealtToTurrets,damageSelfMitigated,deaths,detectorWardsPlaced,doubleKills,dragonKills,firstBloodAssist,firstBloodKill,firstTowerAssist,firstTowerKill,gameEndedInEarlySurrender,gameEndedInSurrender,goldEarned,goldSpent,individualPosition,inhibitorKills,inhibitorsLost,item0,item1,item2,item3,item4,item5,item6,itemsPurchased,killingSprees,kills,lane,largestCriticalStrike,largestKillingSpree,largestMultiKill,longestTimeSpentLiving,magicDamageDealt,magicDamageDealtToChampions,magicDamageTaken,neutralMinionsKilled,nexusKills,nexusLost,objectivesStolen,objectivesStolenAssists,participantId,pentaKills,physicalDamageDealt,physicalDamageDealtToChampions,physicalDamageTaken,profileIcon,puuid,quadraKills,riotIdName,riotIdTagline,role,sightWardsBoughtInGame,spell1Casts,spell2Casts,spell3Casts,spell4Casts,summoner1Casts,summoner1Id,summoner2Casts,summoner2Id,summonerId,summonerLevel,summonerName,teamEarlySurrendered,teamId,teamPosition,timeCCingOthers,timePlayed,totalDamageDealt,totalDamageDealtToChampions,totalDamageShieldedOnTeammates,totalDamageTaken,totalHeal,totalHealsOnTeammates,totalMinionsKilled,totalTimeCCDealt,totalTimeSpentDead,totalUnitsHealed,tripleKills,trueDamageDealt,trueDamageDealtToChampions,trueDamageTaken,turretKills,turretsLost,unrealKills,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win
0,5V5 RANKED SOLO GAMES,Blue,3944288481,13,0,0,8243,11,223,TahmKench,0,7,488.0,1419,488,17878,6,3,0,0,False,False,False,False,False,True,7039,6425,BOTTOM,0,0.0,Doran's Shield,Sunfire Aegis,Plated Steelcaps,Kindlegem,Ruby Crystal,NaN,Stealth Ward,18,0,1,BOTTOM,0,0,1,312,20774,4743,6487,0,0,0.0,0,0,5,0,10610,731,9696,4568,6zpHc103eNNhAt8xC6Ujhak476wF7TuTm4K4vrCU2Zuu9Y...,0,NaN,NaN,CARRY,0,71,4,11,2,3,NaN,4,NaN,-wR2sAu8IN0jiLgjgGyDQsjpnyX5tSwTwZHcIE7L9aS1TbyQ,274,From Iron,False,100,BOTTOM,14,1284,31384,5475,1105,16532,2087,0,98,105,140,1,0,0,0,349,0,0.0,0,16,3,4,9,True
1,5V5 RANKED SOLO GAMES,Blue,3944251280,11,0,0,6578,10,111,Nautilus,0,10,0.0,174,0,16060,5,7,0,0,False,False,False,False,False,False,6099,6000,UTILITY,0,1.0,Plated Steelcaps,Locket of the Iron Solari,Bulwark of the Mountain,Cloth Armor,Warden's Mail,Control Ward,Oracle Lens,28,1,2,BOTTOM,0,2,1,407,10589,4457,4313,0,0,1.0,0,0,5,0,5748,1951,6472,4568,6zpHc103eNNhAt8xC6Ujhak476wF7TuTm4K4vrCU2Zuu9Y...,0,NaN,NaN,SUPPORT,0,50,17,21,9,13,NaN,4,NaN,-wR2sAu8IN0jiLgjgGyDQsjpnyX5tSwTwZHcIE7L9aS1TbyQ,274,From Iron,False,100,UTILITY,46,1347,21927,7026,671,11566,62,0,43,246,98,1,0,5589,616,779,0,9.0,0,53,8,6,31,False
2,5V5 RANKED SOLO GAMES,Red,3944226110,0,0,0,4583,8,202,Jhin,0,6,0.0,0,0,3309,5,2,0,0,False,False,False,False,False,True,4305,3100,BOTTOM,0,0.0,Serrated Dirk,Refillable Potion,Boots,Vampiric Scepter,NaN,Long Sword,Stealth Ward,16,0,1,NONE,358,0,1,281,148,0,1678,0,0,0.0,0,0,9,0,34032,3541,6081,4568,6zpHc103eNNhAt8xC6Ujhak476wF7TuTm4K4vrCU2Zuu9Y...,0,NaN,NaN,SUPPORT,0,36,17,3,10,2,NaN,3,NaN,-wR2sAu8IN0jiLgjgGyDQsjpnyX5tSwTwZHcIE7L9aS1TbyQ,274,From Iron,False,200,BOTTOM,3,912,34180,3541,0,8214,1718,216,99,7,75,3,0,0,0,454,0,3.0,0,18,2,3,10,False
3,5V5 RANKED SOLO GAMES,Red,3941904096,20,0,0,10417,13,111,Nautilus,0,11,320.0,999,320,30781,7,8,0,0,False,False,False,False,False,False,7821,6575,UTILITY,0,0.0,Plated Steelcaps,Locket of the Iron Solari,Bulwark of the Mountain,Control Ward,Kindlegem,Warden's Mail,Oracle Lens,27,0,0,BOTTOM,0,0,0,690,8449,5522,8327,0,0,0.0,0,0,10,0,7874,1919,10809,4568,6zpHc103eNNhAt8xC6Ujhak476wF7TuTm4K4vrCU2Zuu9Y...,0,NaN,NaN,SUPPORT,0,71,46,24,9,5,NaN,5,NaN,-wR2sAu8IN0jiLgjgGyDQsjpnyX5tSwTwZHcIE7L9aS1TbyQ,274,From Iron,False,200,UTILITY,60,1841,21192,8025,1644,19624,1203,0,40,190,212,1,0,4868,584,487,0,1.0,0,64,9,8,35,True
4,5V5 RANKED SOLO GAMES,Red,3941845033,18,0,1,8840,12,111,Nautilus,0,10,796.0,796,796,15092,4,6,0,0,False,False,False,

In [10]:
win_ratio_df = match_data_total_deduped.groupby(['teamPosition', 'championName']).agg({'win': ['sum', 'count']}).droplevel(0, axis = 1)
win_ratio_df['winRatio'] = round(win_ratio_df['sum'] / win_ratio_df['count'], 4)
win_ratio_df = win_ratio_df.reset_index()
win_ratio_df = win_ratio_df.loc[win_ratio_df['teamPosition'] != 'AFK', ]
win_ratio_df.head()

,teamPosition,championName,sum,count,winRatio
4,BOTTOM,Aatrox,0,1,0.0000
5,BOTTOM,Aphelios,15,26,0.5769
6,BOTTOM,Ashe,24,47,0.5106
7,BOTTOM,Brand,2,2,1.0000
8,BOTTOM,Caitlyn,13,30,0.4333


In [11]:
win_ratio_df.groupby('teamPosition').mean()['count']
bottom_avg_game_count = win_ratio_df.groupby('teamPosition').mean()['count']['BOTTOM']
jung_avg_game_count = win_ratio_df.groupby('teamPosition').mean()['count']['JUNGLE']
mid_avg_game_count = win_ratio_df.groupby('teamPosition').mean()['count']['MIDDLE']
top_avg_game_count = win_ratio_df.groupby('teamPosition').mean()['count']['TOP']
supp_avg_game_count = win_ratio_df.groupby('teamPosition').mean()['count']['UTILITY']

In [12]:
bottom_win_rates = win_ratio_df.loc[(win_ratio_df['teamPosition'] == 'BOTTOM') & (win_ratio_df['count'] >= bottom_avg_game_count), ].sort_values(by = 'winRatio', ascending = False).head(10)
jung_win_rates = win_ratio_df.loc[(win_ratio_df['teamPosition'] == 'JUNGLE') & (win_ratio_df['count'] >= jung_avg_game_count), ].sort_values(by = 'winRatio', ascending = False).head(10)
mid_win_rates = win_ratio_df.loc[(win_ratio_df['teamPosition'] == 'MIDDLE') & (win_ratio_df['count'] >= mid_avg_game_count), ].sort_values(by = 'winRatio', ascending = False).head(10)
top_win_rates = win_ratio_df.loc[(win_ratio_df['teamPosition'] == 'TOP') & (win_ratio_df['count'] >= top_avg_game_count), ].sort_values(by = 'winRatio', ascending = False).head(10)
supp_win_rates = win_ratio_df.loc[(win_ratio_df['teamPosition'] == 'UTILITY') & (win_ratio_df['count'] >= supp_avg_game_count), ].sort_values(by = 'winRatio', ascending = False).head(10)

win_rates_top_10 = pd.concat([bottom_win_rates, jung_win_rates, mid_win_rates, top_win_rates, supp_win_rates])
win_rates_top_10['winRatio'] = round((win_rates_top_10['winRatio'] * 100), 1)


In [13]:
def graph_top10_win_rates(data, lane):
    data = data.loc[data['teamPosition'] == lane, ]
    fig = px.bar(data, x = 'championName', y = 'winRatio', text = 'winRatio', color = 'winRatio', color_continuous_scale = px.colors.sequential.Emrld,
                 labels = {'teamPosition': 'Position', 'winRatio': 'Win Percentage', 'championName': 'Champion'},
                 title = f'Top 10 Win Rates for {lane.title()} Position')
    fig.update_layout(
        plot_bgcolor = 'rgb(0,0,0,0)',
        paper_bgcolor = 'rgb(0,0,0,0)',
        width = 1000,
        height = 600,
        title_x = 0.5,
        coloraxis_showscale = False,
        )
    fig.update_xaxes(categoryorder = 'total ascending')
    fig.update_yaxes(ticksuffix = '%', visible = False)
    fig.update_traces(texttemplate = '%{text}%', textposition = 'outside', insidetextanchor = 'end', 
                      textfont_color = '#000000', textfont_size = 14)

    return fig

In [14]:
team_positions = ['BOTTOM', 'UTILITY', 'TOP', 'MIDDLE', 'JUNGLE']
top_10_by_position = []
for position in team_positions:
    fig = graph_top10_win_rates(win_rates_top_10, position)
    top_10_by_position.append(fig)

In [15]:
for fig in top_10_by_position:
    fig.show()